IMPORTAR RECURSOS

In [ ]:
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from functions import *
from getTweets import *
from processTopic import *
from entidades import find_entities
import pandas as pd
import numpy as np
import operator

# **Obtener Tweets**

In [2]:
# Obtener trends
trends = getTrend()
print(trends)

[✓] Trends successfully extracted.
['#CallOutRacismHateSpeech', '#LaMoradaXRPP', '#TheDay_sevENstars_Connected', '#nochedeconfesiones', '#엔하이픈_7개_별들이_연결된_날', 'Arley', 'Austria', 'Barcos', 'Benítez', 'Bosnia', 'Bryan', 'Bustos', 'COLBERT', 'Calcaterra', 'Congreso', 'Corzo', 'Cusco FC', 'ENHYPEN 1ST ANNIVERSARY', 'España', 'Gone Away', 'Guillén', 'Hildebrandt', 'Juan Pablo Varillas', 'La Resistencia', 'Lilia Paredes', 'Liza', 'Lora', 'Maeve', 'Melgar', 'Merino', 'Mitch', 'Mosquera', 'México', 'Otis', 'Percy Rojas', 'Pirata', 'Riquelme', 'Ruby', 'Sporting Cristal', 'THE ENHYPEN DIMENSION OPENS', 'Taylor', 'Tony', 'Trucha', 'Yotun', 'blackpink onu', 'datum', 'godoy', 'juan pari', 'leao', 'raziel garcía']


In [3]:
# # Obtener tweets
# get_tweets_tweepy("juan pari")

# **Crear Noticia**

## Leer trend en json

In [4]:
trend_path = 'tweets\spiderman.json'
trend_name = trend_path.split('\\')[-1].split('.')[0]
data = pd.read_json(trend_path, lines=True)
data.head()

,id,text,screen_name,user_followers,retweet_count,favorite_count,date,language,img
0,1436286323460083712,En 2002 se estrenó la primera película de la s...,manzottipablo,38066,320,1607,2021-09-10 11:11:59,es,http://pbs.twimg.com/ext_tw_video_thumb/143628...
1,1436069507148177408,OFICIAL: #Sony confirma que #InsomniacGames es...,GabyMeza8,228765,213,2783,2021-09-09 20:50:26,es,http://pbs.twimg.com/media/E-3xmmuWQAgLhMo.jpg
2,1437883851909615616,"¡UN ENCUENTRO SEGURO! En entrevista para #IGN,...",GabyMeza8,228765,140,2545,2021-09-14 20:59:59,es,http://pbs.twimg.com/media/E_RjvftWUAYUDLB.jpg
3,1439106087995727872,yo explicandole a @LauChurrupit las peliculas ...,gonzbur,331,0,0,2021-09-18 05:56:43,es,http://pbs.twimg.com/media/E_i7FrKWEAINCFU.jpg
4,1439105999441326080,Nada más espérense a que me llegue mi traje de...,felixmjorge,1784,0,0,2021-09-18 05:56:22,es,http://pbs.twimg.com/media/E_i7RuLVgAIkBKd.jpg


## Limpieza de datos

In [5]:
data_clean = clean_data(trend_path)
data_clean.drop_duplicates(data_clean.columns[~data_clean.columns.isin(['favorite_count'])], keep = 'first')
data_clean.head()

,id,text,screen_name,user_followers,retweet_count,favorite_count,date,language,img
0,1436286323460083712,en se estreno la primera pelicula de la saga...,manzottipablo,38066,320,1607,2021-09-10 11:11:59,es,http://pbs.twimg.com/ext_tw_video_thumb/143628...
1,1436069507148177408,oficial sony confirma que insomniacgames esta ...,GabyMeza8,228765,213,2783,2021-09-09 20:50:26,es,http://pbs.twimg.com/media/E-3xmmuWQAgLhMo.jpg
2,1437883851909615616,un encuentro seguro en entrevista para ign an...,GabyMeza8,228765,140,2545,2021-09-14 20:59:59,es,http://pbs.twimg.com/media/E_RjvftWUAYUDLB.jpg
3,1439106087995727872,yo explicandole a lauchurrupit las peliculas d...,gonzbur,331,0,0,2021-09-18 05:56:43,es,http://pbs.twimg.com/media/E_i7FrKWEAINCFU.jpg
4,1439105999441326080,nada mas esperense a que me llegue mi traje de...,felixmjorge,1784,0,0,2021-09-18 05:56:22,es,http://pbs.twimg.com/media/E_i7RuLVgAIkBKd.jpg


## Palabras de mayor importancia - Count Vectorizer

In [6]:
# Definir stopwords
stop_word_es = get_stop_words('spanish')
def find_top_words(data):
    # contabilizar frecuencias mediante CV
    cv = CountVectorizer(stop_words= stop_word_es)
    cv_matrix = cv.fit_transform(data['text'])
    count_words = np.sum(cv_matrix.toarray(), axis=0)
    words_cv =  {key:count_words[cv.vocabulary_[key]] for key in cv.vocabulary_}
    top_words_cv = sorted(words_cv.items(), key=operator.itemgetter(1), reverse=True)

    # contabilizar frecuencia mediante IDF
    tfidf = TfidfTransformer()
    tfidf_matrix = tfidf.fit_transform(cv_matrix)
    words_idf =  dict(zip(cv.get_feature_names(), tfidf.idf_))
    top_words_idf =  sorted(words_idf.items(), key=operator.itemgetter(1), reverse=True)
    return top_words_cv, top_words_idf

In [7]:
cv, idf = find_top_words(data_clean)

## Clasidicacion mediante glosarios

In [8]:
# Cargar glosarios
glossaries = {}
with os.scandir('glossaries') as gloss_scanIte:
    for dir in gloss_scanIte:
        tmp = open(dir, 'r').readlines()
        lines = [ without_accents(word.rsplit('\n')[0]) for word in tmp]
        name = dir.path.split('\\')[-1].split('.')[0]
        glossaries[name] = lines

# Identificar categoria del trend 
points = {}
words_found = {}
for key in glossaries:
    count = 0
    tmp = []
    for word, score in cv:
        if word in glossaries[key]:
            tmp.append(word)
            count += score
    points[key] = count
    words_found[key] = tmp
category = sorted(points.items(), key=operator.itemgetter(1), reverse=True)[0][0]
category

'entretenimiento'

## Identificacion de titular

In [9]:
data_top_by_favorite = data.sort_values('favorite_count', ascending=False)
data_top_by_favorite.iloc[0,1]

'Spiderman 2 y God of War Ragnarok, suficientes razones para comprarme una ps5 https://t.co/xqxC5ADOPU'

In [10]:
data_top_by_user_followers = data.sort_values('user_followers', ascending=False)
data_top_by_user_followers.iloc[0,1]

'¡WE. ARE. VENOM!\n\n#SpiderMan2 llegará con Miles y Peter enfrentándose a Venom en la esperada secuela del afamado juego de SpiderMan. ¿Están emocionados? Llega en 2023 https://t.co/T4OSFTdeZ3'

In [11]:
data_top_by_retweet_count = data.sort_values('retweet_count', ascending=False)
data_top_by_retweet_count.iloc[0,1]

'Spiderman 2 y God of War Ragnarok, suficientes razones para comprarme una ps5 https://t.co/xqxC5ADOPU'

In [12]:
data_score = data.copy()
data_score['score'] = data_score['user_followers'] * data_score['retweet_count'] * data_score['favorite_count']
data_score_top_by_score = data_score.sort_values('score', ascending=False)
data_score_top_by_score.iloc[0,1]

'OFICIAL: #Sony confirma que #InsomniacGames está trabajando en la secuela del videojuego de #Spiderman de 2018, con #Venom como villano, el juego sera exclusivo para el #PlayStation 🕷️ https://t.co/Z6qAEsMyn8'

## Identificar entidades

In [15]:
# Prepocesar
data_clean_by_enti = data.copy()
data_clean_by_enti['text'] = data_clean_by_enti['text'].apply(clean_text_for_enti)

# entidades
entities = find_entities(data_clean_by_enti)

# procesar entidades
top_entities =  sorted(entities.items(), key=operator.itemgetter(1), reverse=True)[:10]

# Encontrar significado
dic_entities = {}
for entity in top_entities:
    dic_entities[entity[0]] = find_mean(entity[0])

In [16]:
#find_mean('spider man')

## Construccion del .json - output

In [17]:
new = {}
new['title'] = str(data_top_by_user_followers.iloc[0,1])
new['date'] = str(data_top_by_user_followers.iloc[0,6])
new['img'] = str(data_top_by_user_followers.iloc[0,8])
new['category'] = str(category)
new['entities'] = dic_entities

In [18]:
with open(trend_name + '_new.json', 'w', encoding='utf-8') as file:
    json.dump(new, file, indent=4)